In [1]:
import pandas as pd

In [2]:
u18_df = pd.read_csv("U18_3_1.csv")

In [3]:
#列の順番を整理する
#最後にやりたい
# u18_df=u18_df.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','goal_away','goal_home','results_away','results_home','url','time','player']]
# time,playerの列を削除
u18_df=u18_df.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','goal_away','goal_home','results_away','results_home','url']]
u18_df.head(2)

,id,leagu_name,year,month,day,round,team_home,team_away,goal_away,goal_home,results_away,results_home,url
0,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,2,1,http://www.jfa.jp/match/takamado_jfa_u18_premi...
1,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,3,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...


In [4]:
# スコアがNone(=未実施)の試合の行を削除
u18_df=u18_df[u18_df['results_away'] != "None"]
u18_df.tail(3)

,id,leagu_name,year,month,day,round,team_home,team_away,goal_away,goal_home,results_away,results_home,url
73,1504,高円宮杯U-18サッカーリーグ2019,2019,10月,14日,第15節4,ジュビロ磐田U-18,柏レイソルU-18,64分 中島 舜,"61分 清田 奈央弥,90+3分 三木 直土",1,2,http://www.jfa.jp/match/takamado_jfa_u18_premi...
74,1505,高円宮杯U-18サッカーリーグ2019,2019,10月,14日,第15節5,浦和レッドダイヤモンズユース,大宮アルディージャU18,"13分 柴山 昌也,38分 髙田 颯也",None,2,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...
82,1703,高円宮杯U-18サッカーリーグ2019,2019,10月,05日,第17節3,流通経済大学付属柏高校,市立船橋高校,"14分 鈴木 唯人,19分 鈴木 唯人",40分 森山 一斗,2,1,http://www.jfa.jp/match/takamado_jfa_u18_premi...


In [5]:
#awayチームの得点者とその試合のidのみを抜き出す
df_spr_away = pd.DataFrame(u18_df['goal_away'].str.split(',', expand=True))
df_spr_away["id"] =u18_df["id"]

df_spr_away.head()

,0,1,2,3,id
0,26分 渡邊 翔耶,90+4分 野中 悠翔,None,None,101
1,25分 染野 唯月,79分 染野 唯月,85分 染野 唯月,None,102
2,17分 青島 太一,81分 青島 太一,None,None,103
3,32分 後藤 健太,86分 古澤 ナベル慈宇,None,None,104
4,None,None,None,None,105


In [6]:
# 先ほどの表から、1ゴール目のデータだけの表を作り、ゴールのない試合は削除
df_spr_away0 = pd.DataFrame({
    "goal":df_spr_away[0],
    "id":df_spr_away["id"]
})
df_spr_away0=df_spr_away0[df_spr_away0['goal'] != "None"] # df_spr_away0.dropna(inplace=True) はもともと"None"なので、使えない
df_spr_away0.head(3)

,goal,id
0,26分 渡邊 翔耶,101
1,25分 染野 唯月,102
2,17分 青島 太一,103


In [7]:
#2ゴール目以降のデータも追加
for repeat in range(1, ( (len(df_spr_away.columns)) -1) ):
    df_spr_away_n = pd.DataFrame({
    "goal":df_spr_away[repeat],
    "id":df_spr_away["id"]
    })
    df_spr_away_n.dropna(inplace=True)
    df_spr_away0 = pd.concat([df_spr_away0,df_spr_away_n],sort=True,copy=False,ignore_index=True)

df_spr_away0.shape

(99, 2)

In [8]:
#選手名と時間を分ける
# df3 = pd.concat([df['local'], df['domain'].str.split('.', expand=True)], axis=1)
df_spr_away0 = pd.concat([df_spr_away0,df_spr_away0["goal"].str.split("分",expand=True)],axis=1).drop("goal", axis=1)
df_spr_away0.rename(columns={0: 'time', 1: 'player'}, inplace=True)
df_spr_away0.head()

,id,time,player
0,101,26,渡邊 翔耶
1,102,25,染野 唯月
2,103,17,青島 太一
3,104,32,後藤 健太
4,202,55,福井 啓太


In [9]:
#並び変える（ソート）
#最後にやりたい
df_spr_away0=df_spr_away0.sort_values(["id","time"])
df_spr_away0.head()

,id,time,player
0,101,26,渡邊 翔耶
57,101,90+4,野中 悠翔
1,102,25,染野 唯月
58,102,79,染野 唯月
90,102,85,染野 唯月


In [10]:
#試合情報を結合
df_away = pd.merge(df_spr_away0,u18_df, on="id")
df_away=df_away.loc[:,['id','leagu_name','year','month','day','round','team_home','team_away','goal_away','goal_home','results_away','results_home','url','time','player']]

df_away

,id,leagu_name,year,month,day,round,team_home,team_away,goal_away,goal_home,results_away,results_home,url,time,player
0,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,2,1,http://www.jfa.jp/match/takamado_jfa_u18_premi...,26,渡邊 翔耶
1,101,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節1,市立船橋高校,ジュビロ磐田U-18,"26分 渡邊 翔耶,90+4分 野中 悠翔",52分 石田 侑資,2,1,http://www.jfa.jp/match/takamado_jfa_u18_premi...,90+4,野中 悠翔
2,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,3,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...,25,染野 唯月
3,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,3,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...,79,染野 唯月
4,102,高円宮杯U-18サッカーリーグ2019,2019,04月,06日,第1節2,柏レイソルU-18,尚志高校,"25分 染野 唯月,79分 染野 唯月,85分 染野 唯月",None,3,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...,85,染野 唯月
5,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,"17分 青島 太一,81分 青島 太一",None,2,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...,17,青島 太一
6,103,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節3,浦和レッドダイヤモンズユース,清水エスパルスユース,"17分 青島 太一,81分 青島 太一",None,2,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...,81,青島 太一
7,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,2,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...,32,後藤 健太
8,104,高円宮杯U-18サッカーリーグ2019,2019,04月,07日,第1節4,流通経済大学付属柏高校,青森山田高校,"32分 後藤 健太,86分 古澤 ナベル慈宇",None,2,0,http://www.jfa.jp/match/takamado_jfa_u18_premi...,86,古澤 ナベル慈宇
9,202,高円宮杯U-18サッカーリーグ2019,2019,04月,14日,第2節2,市立船橋高校,大宮アルディージャU18,55分 福井 啓太,"45+2分 植松 建斗,69分 加藤 想音,80分 森 英希",1,3,http://www.jfa.jp/match/takamado_jfa_u18_premi...,55,福井 啓太


In [14]:
# CSV出力
df_away.to_csv("./u18_19_away.csv", 
          index=False   # 行インデックスを削除
         )